<!-- Assignment X- WS 2022 -->

# Regular-, Initial- &amp; Normalisation (10 points)

# This notebook contains the sixth assignment for the exercises in Deep Learning and Neural Nets 1.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility function that should work without problems.
Please, do not alter this code or add extra import statements in your submission, unless it is explicitly requested!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, the goal is to get familiar with some tools that can help to speed up the training process of neural networks. **Regularisation** is a technique that can be used to avoid overfitting. Knowing what kind of **initialisation** to use in what context is often important to assure fast learning. **Normalisation** is a tool that tackles the problem of drifting distributions that pops up in very deep networks and hinders learning.

In [1]:
import numpy as np

from nnumpy import Module, Parameter
from nnumpy.testing import gradient_check

rng = np.random.default_rng(1856)

In [2]:
def initialiser(fn):
    """ 
    Function decorator for initialisation functions that
    enables initialisation of multiple weight arrays at once. 
    """
    
    def init_wrapper(*parameters, **kwargs):
        for par in parameters:
            par[:] = fn(par.shape, **kwargs)
            par.zero_grad()
    
    init_wrapper.__name__ = fn.__name__ + "_init"
    init_wrapper.__doc__ = fn.__doc__
    return init_wrapper

## Regularisation

Neural networks are infamously prone to overfitting. Just as with any machine learning model, overfitting can relatively easily be detected by monitoring the learning curves on training and validation sets. In order to counter these effects, you can use regularisation techniques. 

![learning curves](https://d2l.ai/_images/capacity-vs-error.svg)

One possibility is to use well-known approaches from regression: e.g. $L_1$ or $L_2$ regularisation, which are also known as *LASSO*, resp. *ridge* regression. Also simply interrupting the learning before the overfitting occurs can prevent overfitting models. These are only a few examples, but most regularisation techniques are not exlusive to neural networks. However, there is one NN-exclusive approach that is very commonly used: **Dropout**.

### Exercise 1: Dropout (3 Points)

Dropout is a simple, but very effective regularisation technique that can be added practically anywhere in a network. The idea of dropout is to randomly disable a few neurons during training. During inference all neurons are used. Since this would lead to a shift in distribution of the pre-activations in the next layer (training vs inference), the neurons are scaled down during evaluation so that the distributions during inference and training are approximately the same. In order to avoid the need to change the network during evaluation, it is also possible to scale up the activations during training. This specific change in implementation is often referred to as *inverted* dropout.

> Implement the forward and backward pass of an **inverted dropout** module.

**Hint:** use the `Module` attribute `predicting` to check whether you are in prediction or training or mode.

In [3]:
class Dropout(Module):
    """ NNumpy implementation of (inverted) dropout. """

    def __init__(self, rate: float = .5, seed: int = None):
        """
        Parameters
        ----------
        rate : float, optional
            The percentage of neurons to be dropped.
        seed : int, optional
            Seed for the pseudo random generator.
        """
        super().__init__()
        if rate < 0. or rate > 1.:
            raise ValueError("dropout rate should be between zero and one")

        self.rate = float(rate)
        self.rng = np.random.default_rng(seed)

    def compute_outputs(self, x):
        # YOUR CODE HERE
        if(self.predicting):
            # predicting mode / testing mode
            return x, None
        else:
            # training mode
            self.mask = (self.rng.random(x.shape) < self.rate) / (1 - self.rate)
            out = x * self.mask
            return out, self.mask

    def compute_grads(self, grads, multiplier):
        # YOUR CODE HERE
        if(self.predicting):
            #predicting mode / testing mode
            return grads
        else:
            #training mode
            return grads * multiplier

In [4]:
# Test Cell: do not edit or delete!
dropout_layer = Dropout(rate=0.5)
x = np.ones(7)
y, cache = dropout_layer.compute_outputs(x)
assert isinstance(y, np.ndarray), (
    "ex1: the output of Dropout.compute_outputs is not a numpy array (-0.5 points)"
)
assert y.shape == x.shape, (
    "ex1: the output of Dropout.compute_outputs has incorrect shape (-0.5 points)"
)

In [5]:
# Test Cell: do not edit or delete!

In [6]:
# Test Cell: do not edit or delete!

In [7]:
# Test Cell: do not edit or delete!
dropout_layer = Dropout(rate=0.2)
x = rng.standard_normal(size=(1, 11, 13))
assert gradient_check(dropout_layer, x, debug=True), (
    "ex1: gradient check for Dropout failed (-1 point)"
)

## Initialisation

A good initialisation has proven to be very important to learn deep neural networks. Although this can be considered as a well-known fact, it is astonishing how often initialisation is ignored. Since simply initialising all parameters with some constant does not work, the initial values are generally small, randomly generated numbers. There are different distributions to sample these values from, however.

### Exercise 2: Xavier Glorot (3 Points)

When generating random values, there are different choices for the distribution to draw numbers from. The uniform or Gaussian (a.k.a. normal) distributions are most common for initialising the parameters of a neural network. After all, these are simple distributions that can easily be centred around zero.

Apart from centring the initial parameters around zero, it is also helpful to make sure that the weights have a specific amount of variance. Xavier Glorot proposed to use the reciprocal of the average of fan-in and fan-out, i.e. $\frac{2}{\text{fan-in} + \text{fan-out}}$, for the variance. Here, *fan-in* and *fan-out* are the number of incoming connections per output neuron and number of outgoing connections per input neuron, respectively.

Note, however, that this proposal only holds for identity and $\tanh$ activation functions. When using different activation functions, the variance of the initial parameters need to be scaled correspondingly. This can be done by means of a linear *gain* factor that accounts for the effect of the activation functions.

 > Implement the `glorot_uniform` function so that it produces initial weights for a parameter with given shape according to the proposal from Xavier Glorot. Make sure to make use of the seed for the initialisation, as well as the `gain` parameter.
 
**Hint:** Think carefully about the number of connections in convolutional layers.

In [8]:
@initialiser
def glorot_uniform(shape, gain: float = 1., seed: int = None):
    """
    Initialise parameter cf. Glorot, using a uniform distribution.
    
    Parameters
    ----------
    shape : tuple
        The shape of the parameter to be initialised.
    gain : float, optional
        Multiplier for the variance of the initialisation.
    seed : int, optional
        Seed for generating pseudo random numbers.
        
    Returns
    -------
    values: ndarray
        Numpy array with the initial weight values
        with dimensions as specified by `shape`.
    """
    # YOUR CODE HERE
    rng = np.random.default_rng(seed)

    fan_in, fan_out = compute_fans(shape)

    # Calculate the standard deviation based on Glorot initialization
    std_dev = gain * np.sqrt(2.0 / (fan_in + fan_out))

    # Initialize weights with uniform distribution
    values = rng.uniform(-std_dev, std_dev, size=shape)

    return values

def compute_fans(shape):
    """
    Compute the number of input and output units for a weight tensor.
    
    Parameters
    ----------
    shape : tuple
        The shape of the weight tensor.
        
    Returns
    -------
    fan_in : int
        Number of input units.
    fan_out : int
        Number of output units.
    """
    if len(shape) == 2:  # For dense layers
        fan_in, fan_out = shape[0], shape[1]
    elif len(shape) in {3, 4}:  # For convolutional layers
        fan_in = np.prod(shape[:-1])
        fan_out = shape[-1]
    else:
        raise ValueError("Unsupported shape for Glorot initialization")

    return fan_in, fan_out

In [9]:
# Test Cell: do not edit or delete!
par = Parameter(np.ones((10, 10)))
glorot_uniform(par, seed=1806)
assert np.all(par != 1), (
    "ex2: glorot_uniform function does not return (reasonable) values (-0.5 points)"
)

In [10]:
# Test Cell: do not edit or delete!
par1 = Parameter(np.ones((10, 10)))
par2 = Parameter(np.ones((100, 100)))
glorot_uniform(par1, par2, seed=1806)
assert not np.isclose(par1.var(), par2.var(), atol=1e-3), (
    "ex2: glorot_uniform function does not (correctly) use FC parameter shape (-0.5 points)"
)
par1 = Parameter(np.ones((10, 10, 3, 3)))
par2 = Parameter(np.ones((100, 100, 3, 3)))
glorot_uniform(par1, par2, seed=1806)
assert not np.isclose(par1.var(), par2.var(), atol=1e-3), (
    "ex2: glorot_uniform function does not (correctly) use Conv parameter shape (-0.5 points)"
)

In [11]:
# Test Cell: do not edit or delete!

In [12]:
# Test Cell: do not edit or delete!

## Normalisation

Whereas initialisation ensures proper variance propagation through the network when learning starts, it does not ensure that the weights keep these properties after some updates. To ensure a steady flow of information through the network, normalisation techniques were introduced. The idea of normalisation is to normalise either the activations or pre-activations. This can be done explicitly, using techniques like *Batch* or *Layer Normalisation*, or more implicitly, e.g. *weight normalisation* or using *self-normalising networks*.

### Exercise 3: Batch Normalisation (4 Points)

Batch Normalisation (or *batch norm* for short) has empirically proven to be a very useful technique for improving the performance of neural networks. It is not quite clear why it works so well, but there is some form of consensus that it acts as a regulariser and improves gradient flow in the network. 

The core principle of batch norm is to subtract the mean and divide by the standard deviation of the data, computed over the samples in one batch. Each neuron is normalised individually, so that all neurons have zero mean and unit variance. Batch norm also uses parameters $\gamma$ and $\beta$ to scale, resp. shift the normalised signal. Note that, since batch norm relies on batch statistics, it requires a large batch size to work properly!

During inference, it is not uncommon to want a prediction for a single sample. Therefore, you generally do not want to use the mean computed during inference. Therefore, batch norm tracks the statistics of the data during training using a [moving average](https://en.wikipedia.org/wiki/Moving_average). During evaluation, these tracked statistics, i.e. the statistics of the training data, are used to normalise the previously unseen samples.

 > Implement the forward and backward pass of the batch normalisation module. Use a simple moving average for tracking the statistics.
 
**Hint:** You can track the statistics in attributes of the module.

In [13]:
class BatchNormalisation(Module):
    """ NNumpy implementation of batch normalisation. """

    def __init__(self, dims: tuple, eps: float = 1e-8):
        """
        Parameters
        ----------
        dims : tuple of ints
            The shape of the incoming signal (without batch dimension).
        eps : float, optional
            Small value for numerical stability.
        """
        super().__init__()
        self.dims = tuple(dims)
        self.eps = float(eps)
        
        self.gamma = self.register_parameter('gamma', np.ones(self.dims))
        self.beta = self.register_parameter('beta', np.zeros(self.dims))
        
        self.running_count = 0
        self.running_stats = np.zeros((2, ) + self.dims)

    def compute_outputs(self, x):
        mean = np.mean(x, axis=0)
        var = np.var(x, axis=0)

        x_normalized = (x - mean) / np.sqrt(var + self.eps)
        out = self.gamma * x_normalized + self.beta

        # Update running statistics during training
        if not self.predicting:
            self.running_count += 1
            decay = 0.9  # Adjust decay factor as needed
            self.running_stats[0] = decay * self.running_stats[0] + (1 - decay) * mean
            self.running_stats[1] = decay * self.running_stats[1] + (1 - decay) * var

        return out, (x, mean, var, x_normalized)


    def compute_grads(self, grads, cache):
        x, mean, var, x_normalized = cache

        N = x.shape[0]
        dx_normalized = grads * self.gamma
        dvar = np.sum(dx_normalized * (x - mean), axis=0) * -0.5 * np.power(var + self.eps, -1.5)
        dmean = np.sum(dx_normalized * -1 / np.sqrt(var + self.eps), axis=0) + dvar * np.sum(-2 * (x - mean), axis=0) / N
        dx = dx_normalized / np.sqrt(var + self.eps) + dvar * 2 * (x - mean) / N + dmean / N

        dgamma = np.sum(grads * x_normalized, axis=0)
        dbeta = np.sum(grads, axis=0)

        self.gamma.grad = dgamma
        self.beta.grad = dbeta

        return dx

In [14]:
# Test Cell: do not edit or delete!
x = np.linspace(-1, 3, 50).reshape(10, 5)
bn = BatchNormalisation(x.shape[1:])
y_train, _ = bn.compute_outputs(x)
assert isinstance(y_train, np.ndarray), (
    "ex3: output of BatchNormalisation.compute_outputs is not a numpy array (-0.5 points)"
)
assert y_train.shape == x.shape, (
    "ex3: output of BatchNormalisation.compute_outputs has incorrect shape (-0.5 points)"
)

bn.eval()
y_eval, _ = bn.compute_outputs(x)
assert isinstance(y_eval, np.ndarray), (
    "ex3: output of BatchNormalisation.compute_outputs is not a numpy array in eval mode (-0.5 points)"
)
assert y_eval.shape == x.shape, (
    "ex3: output of BatchNormalisation.compute_outputs has incorrect shape in eval mode (-0.5 points)"
)

In [15]:
# Test Cell: do not edit or delete!
x = np.linspace(-1, 3, 50).reshape(10, 5)
bn = BatchNormalisation(x.shape[1:])
y_train, _ = bn.compute_outputs(x)
assert np.isclose(y_train.mean(), 0.), (
    "ex3: BatchNormalisation.compute_outputs does not produce zero-mean outputs (-1 point)"
)
assert np.isclose(y_train.var(), 1.), (
    "ex3: BatchNormalisation.compute_outputs does not produce unit variance outputs (-1 point)"
)

bn.eval()
y_eval, _ = bn.compute_outputs(x)
assert np.isclose(y_eval.mean(), 0.), (
    "ex3: BatchNormalisation.compute_outputs does not produce zero-mean outputs in eval mode (-1 point)"
)
assert np.isclose(y_eval.var(), 1.), (
    "ex3: BatchNormalisation.compute_outputs does not produce unit variance outputs in eval mode(-1 point)"
)

In [16]:
# Test Cell: do not edit or delete!

In [17]:
# Test Cell: do not edit or delete!

In [18]:
# Test Cell: do not edit or delete!
shape_ = (7, 3, 5)
x = rng.uniform(0, 3, size= shape_)
bn = BatchNormalisation(x.shape[1:])
assert gradient_check(bn, x, debug=True), (
    "ex3: gradient check for BatchNormalization failed (-1.5 points)"
)